In [1]:
import tensorflow as tf
import numpy as np
from time import time
import os
print(tf.__version__)

2022-05-02 16:36:44.750140: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


2.3.0


### Checkpoint function

In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")
    
    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt:
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split("-")[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

### Data load & pre-processing function

In [3]:
def load_mnist():
    (train_data, train_labels), (test_data, test_labels) = tf.keras.datasets.mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1)  # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1)  # [N, 28, 28] -> [N, 28, 28, 1]
    
    train_data, test_data = normalize(train_data, test_data)
    
    train_labels = tf.keras.utils.to_categorical(train_labels, 10)  # [N, ] -> [N, 10]
    test_labels = tf.keras.utils.to_categorical(test_labels, 10)  # [N, ] -> [N, 10]
    
    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0
    
    return train_data, test_data

### Performance function

In [4]:
def loss_function(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits,
                                                                  y_true=labels,
                                                                  from_logits=True))
    return loss

def accuracy_function(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_function(model, images, labels)
    return tape.gradient(loss, model.trainable_variables)

### Create model (class version)

In [5]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()
        
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Flatten())
        
        for i in range(4):
            self.model.add(tf.keras.layers.Dense(units=512, use_bias=True, kernel_initializer=weight_init))
            self.model.add(tf.keras.layers.BatchNormalization())
            self.model.add(tf.keras.layers.Activation(tf.keras.activations.relu))
        
        self.model.add(tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init))
    
    def call(self, x, training=None, mask=None):
        x = self.model(x)
        return x

### Define data & hyper-parameter

In [6]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
        shuffle(buffer_size=100000).\
        prefetch(buffer_size=batch_size).\
        batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
        shuffle(buffer_size=100000).\
        prefetch(buffer_size=len(test_x)).\
        batch(len(test_x))

2022-05-02 16:36:46.391868: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-05-02 16:36:46.464251: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:03:00.0 name: GeForce RTX 2080 computeCapability: 7.5
coreClock: 1.71GHz coreCount: 46 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.23GiB/s
2022-05-02 16:36:46.464859: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 1 with properties: 
pciBusID: 0000:04:00.0 name: GeForce RTX 2080 computeCapability: 7.5
coreClock: 1.71GHz coreCount: 46 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.23GiB/s
2022-05-02 16:36:46.464880: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-05-02 16:36:46.466511: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2022-05

### Define model & optimizer & writer

In [7]:
""" Model """
network = create_model_class(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = "checkpoints"
logs_dir = "logs"

model_dir = "nn_batchnorm"

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

### Restore checkpoint & start train or test phase

In [8]:
if train_flag:
    checkpoint = tf.train.Checkpoint(dnn=network)
    
    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()
    
    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)
    
    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)
        counter = checkpoint-counter
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.trainable_variables))
                
                train_loss = loss_function(network, train_input, train_label)
                train_accuracy = accuracy_function(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:
                    test_accuracy = accuracy_function(network, test_input, test_label)
                
                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + "-{}".format(counter))

# test phase
else:
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:
        test_accuracy = accuracy_function(network, test_input, test_label)
    
    print("test_Accuracy : %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.8408, train_loss: 0.35232943, train_accuracy: 0.3047, test_Accuracy: 0.1839
Epoch: [ 0] [    1/  468] time: 1.1276, train_loss: 0.31671065, train_accuracy: 0.3984, test_Accuracy: 0.3161
Epoch: [ 0] [    2/  468] time: 1.3976, train_loss: 0.33030578, train_accuracy: 0.4453, test_Accuracy: 0.4050
Epoch: [ 0] [    3/  468] time: 1.6669, train_loss: 0.27693295, train_accuracy: 0.3750, test_Accuracy: 0.4578
Epoch: [ 0] [    4/  468] time: 1.9493, train_loss: 0.28389019, train_accuracy: 0.5078, test_Accuracy: 0.5025
Epoch: [ 0] [    5/  468] time: 2.2151, train_loss: 0.40113661, train_accuracy: 0.5391, test_Accuracy: 0.5522
Epoch: [ 0] [    6/  468] time: 2.4839, train_loss: 0.27185813, train_accuracy: 0.6562, test_Accuracy: 0.6038
Epoch: [ 0] [    7/  468] time: 2.7590, train_loss: 0.24482886, train_accuracy: 0.6719, test_Accuracy: 0.6817
Epoch: [ 0] [    8/  468] time: 3.0669,

Epoch: [ 0] [   74/  468] time: 21.4725, train_loss: 0.10994487, train_accuracy: 0.9297, test_Accuracy: 0.9248
Epoch: [ 0] [   75/  468] time: 21.7515, train_loss: 0.18384221, train_accuracy: 0.8750, test_Accuracy: 0.9227
Epoch: [ 0] [   76/  468] time: 22.0327, train_loss: 0.20732856, train_accuracy: 0.9453, test_Accuracy: 0.9215
Epoch: [ 0] [   77/  468] time: 22.2999, train_loss: 0.09516905, train_accuracy: 0.9453, test_Accuracy: 0.9211
Epoch: [ 0] [   78/  468] time: 22.5732, train_loss: 0.08606665, train_accuracy: 0.9297, test_Accuracy: 0.9258
Epoch: [ 0] [   79/  468] time: 22.8375, train_loss: 0.07009915, train_accuracy: 0.9844, test_Accuracy: 0.9285
Epoch: [ 0] [   80/  468] time: 23.1355, train_loss: 0.10702958, train_accuracy: 0.9609, test_Accuracy: 0.9329
Epoch: [ 0] [   81/  468] time: 23.4089, train_loss: 0.07798913, train_accuracy: 0.9531, test_Accuracy: 0.9347
Epoch: [ 0] [   82/  468] time: 23.6933, train_loss: 0.07753186, train_accuracy: 0.9531, test_Accuracy: 0.9370
E

Epoch: [ 0] [  148/  468] time: 41.7329, train_loss: 0.08718719, train_accuracy: 0.9531, test_Accuracy: 0.9540
Epoch: [ 0] [  149/  468] time: 42.0221, train_loss: 0.10274702, train_accuracy: 0.9688, test_Accuracy: 0.9546
Epoch: [ 0] [  150/  468] time: 42.2981, train_loss: 0.07203773, train_accuracy: 0.9688, test_Accuracy: 0.9552
Epoch: [ 0] [  151/  468] time: 42.5739, train_loss: 0.10487464, train_accuracy: 0.9609, test_Accuracy: 0.9549
Epoch: [ 0] [  152/  468] time: 42.8566, train_loss: 0.11816280, train_accuracy: 0.9531, test_Accuracy: 0.9551
Epoch: [ 0] [  153/  468] time: 43.1267, train_loss: 0.10053615, train_accuracy: 0.9609, test_Accuracy: 0.9555
Epoch: [ 0] [  154/  468] time: 43.3912, train_loss: 0.17058930, train_accuracy: 0.9141, test_Accuracy: 0.9568
Epoch: [ 0] [  155/  468] time: 43.6699, train_loss: 0.12620495, train_accuracy: 0.9453, test_Accuracy: 0.9577
Epoch: [ 0] [  156/  468] time: 43.9616, train_loss: 0.07013953, train_accuracy: 0.9844, test_Accuracy: 0.9581
E

Epoch: [ 0] [  222/  468] time: 61.7778, train_loss: 0.09845211, train_accuracy: 0.9453, test_Accuracy: 0.9558
Epoch: [ 0] [  223/  468] time: 62.0640, train_loss: 0.12847172, train_accuracy: 0.9531, test_Accuracy: 0.9563
Epoch: [ 0] [  224/  468] time: 62.3481, train_loss: 0.03387771, train_accuracy: 0.9688, test_Accuracy: 0.9571
Epoch: [ 0] [  225/  468] time: 62.6228, train_loss: 0.05454326, train_accuracy: 0.9922, test_Accuracy: 0.9558
Epoch: [ 0] [  226/  468] time: 62.8865, train_loss: 0.10825878, train_accuracy: 0.9297, test_Accuracy: 0.9563
Epoch: [ 0] [  227/  468] time: 63.1439, train_loss: 0.09103535, train_accuracy: 0.9688, test_Accuracy: 0.9561
Epoch: [ 0] [  228/  468] time: 63.4319, train_loss: 0.06984237, train_accuracy: 0.9766, test_Accuracy: 0.9560
Epoch: [ 0] [  229/  468] time: 63.7031, train_loss: 0.09846344, train_accuracy: 0.9531, test_Accuracy: 0.9559
Epoch: [ 0] [  230/  468] time: 63.9649, train_loss: 0.07855380, train_accuracy: 0.9688, test_Accuracy: 0.9562
E

Epoch: [ 0] [  296/  468] time: 82.2309, train_loss: 0.10133335, train_accuracy: 0.9688, test_Accuracy: 0.9630
Epoch: [ 0] [  297/  468] time: 82.5121, train_loss: 0.09930135, train_accuracy: 0.9531, test_Accuracy: 0.9628
Epoch: [ 0] [  298/  468] time: 82.7770, train_loss: 0.12983106, train_accuracy: 0.9531, test_Accuracy: 0.9633
Epoch: [ 0] [  299/  468] time: 83.0416, train_loss: 0.16157368, train_accuracy: 0.9609, test_Accuracy: 0.9627
Epoch: [ 0] [  300/  468] time: 83.3244, train_loss: 0.10563606, train_accuracy: 0.9688, test_Accuracy: 0.9629
Epoch: [ 0] [  301/  468] time: 83.5787, train_loss: 0.09481158, train_accuracy: 0.9688, test_Accuracy: 0.9629
Epoch: [ 0] [  302/  468] time: 83.8556, train_loss: 0.09434139, train_accuracy: 0.9688, test_Accuracy: 0.9624
Epoch: [ 0] [  303/  468] time: 84.1189, train_loss: 0.04155288, train_accuracy: 0.9844, test_Accuracy: 0.9620
Epoch: [ 0] [  304/  468] time: 84.3810, train_loss: 0.12011297, train_accuracy: 0.9297, test_Accuracy: 0.9623
E

Epoch: [ 0] [  370/  468] time: 103.0694, train_loss: 0.08397372, train_accuracy: 0.9688, test_Accuracy: 0.9686
Epoch: [ 0] [  371/  468] time: 103.3368, train_loss: 0.04291859, train_accuracy: 0.9766, test_Accuracy: 0.9688
Epoch: [ 0] [  372/  468] time: 103.6166, train_loss: 0.06071380, train_accuracy: 0.9688, test_Accuracy: 0.9686
Epoch: [ 0] [  373/  468] time: 103.8938, train_loss: 0.06091554, train_accuracy: 0.9922, test_Accuracy: 0.9688
Epoch: [ 0] [  374/  468] time: 104.1675, train_loss: 0.06425814, train_accuracy: 0.9766, test_Accuracy: 0.9699
Epoch: [ 0] [  375/  468] time: 104.4452, train_loss: 0.03476021, train_accuracy: 0.9844, test_Accuracy: 0.9694
Epoch: [ 0] [  376/  468] time: 104.7050, train_loss: 0.07710990, train_accuracy: 0.9922, test_Accuracy: 0.9686
Epoch: [ 0] [  377/  468] time: 104.9593, train_loss: 0.04115514, train_accuracy: 0.9844, test_Accuracy: 0.9687
Epoch: [ 0] [  378/  468] time: 105.2098, train_loss: 0.04708014, train_accuracy: 0.9922, test_Accuracy:

Epoch: [ 0] [  444/  468] time: 123.5852, train_loss: 0.05766146, train_accuracy: 0.9688, test_Accuracy: 0.9553
Epoch: [ 0] [  445/  468] time: 123.8640, train_loss: 0.08620204, train_accuracy: 0.9531, test_Accuracy: 0.9547
Epoch: [ 0] [  446/  468] time: 124.1549, train_loss: 0.02220894, train_accuracy: 0.9922, test_Accuracy: 0.9529
Epoch: [ 0] [  447/  468] time: 124.4241, train_loss: 0.09801264, train_accuracy: 0.9609, test_Accuracy: 0.9533
Epoch: [ 0] [  448/  468] time: 124.7181, train_loss: 0.07369795, train_accuracy: 0.9688, test_Accuracy: 0.9529
Epoch: [ 0] [  449/  468] time: 124.9915, train_loss: 0.06143732, train_accuracy: 0.9531, test_Accuracy: 0.9547
Epoch: [ 0] [  450/  468] time: 125.2783, train_loss: 0.06512268, train_accuracy: 0.9688, test_Accuracy: 0.9566
Epoch: [ 0] [  451/  468] time: 125.5617, train_loss: 0.04307582, train_accuracy: 0.9766, test_Accuracy: 0.9598
Epoch: [ 0] [  452/  468] time: 125.8247, train_loss: 0.06821468, train_accuracy: 0.9609, test_Accuracy: